# COVIDvu <img src='resources/UN-flag.png' align='right'>

COVID-19 view various charts.

---
## Runtime prerequisites

In [ ]:
%%capture --no-stderr requirementsOutput

displayRequirementsOutput = False

%pip install -r requirements.txt
from utils import autoReloadCode; autoReloadCode()

In [ ]:
if displayRequirementsOutput:
    requirementsOutput.show()

---
## Pull latest datasets from the JH CSSE GitHub repository

This function is `crontab` ready; it can be called from here, the command line, or as a system job.

In [ ]:
%sx ./refreshdata

---
## Confirmed, deaths, recovered datasets

In [ ]:
import os

import numpy as np
import pandas as pd

import utils
import vujson

pd.options.mode.chained_assignment = None

# Confirm input file definitions:
( vujson.JH_CSSE_FILE_CONFIRMED, vujson.JH_CSSE_FILE_DEATHS, vujson.JH_CSSE_FILE_RECOVERED, )

In [ ]:
confirmedCasesTrim     = vujson.allCases()
confirmedDeathsTrim    = vujson.allCases(vujson.JH_CSSE_FILE_DEATHS)
confirmedRecoveredTrim = vujson.allCases(vujson.JH_CSSE_FILE_RECOVERED)

In [ ]:
unknownOutcomes = confirmedCasesTrim - confirmedDeathsTrim - confirmedRecoveredTrim

In [ ]:
mortalityRate = confirmedDeathsTrim/confirmedCasesTrim

In [ ]:
recoveryRate = confirmedRecoveredTrim/confirmedCasesTrim

In [ ]:
knownOutcomeRate = (confirmedCasesTrim - unknownOutcomes)/confirmedCasesTrim

---
## Interactive plots

In [ ]:
import visualize
from ipywidgets import widgets
from ipywidgets import interact
from ipywidgets import fixed

In [ ]:
countries = list(confirmedCasesTrim.columns)
multiCountry = widgets.SelectMultiple(
    options=countries,
    value=['!Global', '!Outside Mainland China'],
    description='Country',
    disabled=False
)
log = widgets.Checkbox(value=False, description='Log scale')

### Confirmed cases

In [ ]:
interact(visualize.plotTimeSeriesInteractive, 
         df=fixed(confirmedCasesTrim), 
         selectedColumns=multiCountry, 
         log=log,
         yLabel=fixed('Total confirmed cases'),
         title=fixed('COVID-19 total confirmed cases')
);

In [ ]:
interact(visualize.plotTimeSeriesInteractive, 
         df=fixed(confirmedCasesTrim.diff(axis=0)), 
         selectedColumns=multiCountry, 
         log=log,
         yLabel=fixed('Confirmed cases (per day)'),
         title=fixed('COVID-19 confirmed new cases (per day)')
);

### Confirmed deaths

In [ ]:
interact(visualize.plotTimeSeriesInteractive, 
         df=fixed(confirmedDeathsTrim), 
         selectedColumns=multiCountry, 
         log=log,
         yLabel=fixed('Number of deaths'),
         title=fixed('COVID-19 total confirmed deaths')
);

### Untracked cases

In [ ]:
interact(visualize.plotTimeSeriesInteractive, 
         df=fixed(unknownOutcomes), 
         selectedColumns=multiCountry, 
         log=log,
         yLabel=fixed('Total unknown outcomes'),
         title=fixed('COVID-19 unknown outcomes from confirmed cases')
);

---
## Mortality and recovery rates

### Mortality rate

In [ ]:
interact(visualize.plotTimeSeriesInteractive, 
         df=fixed(mortalityRate), 
         selectedColumns=multiCountry, 
         log=log,
         yLabel=fixed('Mortality rate (deaths/cases)'),
         title=fixed('COVID-19 mortality rate')
);

### Plot recovery rate

In [ ]:
interact(visualize.plotTimeSeriesInteractive, 
         df=fixed(recoveryRate), 
         selectedColumns=multiCountry, 
         log=log,
         yLabel=fixed('Recovery rate (recovered/cases)'),
         title=fixed('COVID-19 recovery rate')
);

### Plot reporting rate

In [ ]:
interact(visualize.plotTimeSeriesInteractive, 
         df=fixed(knownOutcomeRate), 
         selectedColumns=multiCountry, 
         log=log,
         yLabel=fixed('Known outcome rate'),
         title=fixed('COVID-19 known outcome rate')
);

---
&#169; the COVIDvu Contributors.  All rights reserved.